This training run is on the initial dataset given by Davut, (named: airfoil_dataset_64pts_200foils.csv). The model is being run using an NIF model.

In [1]:
%load_ext autoreload
%autoreload 2

# Define autroreload so that it doesn't cause pain in the ass when we change the functions and run this notebook

In [2]:
# Import all the models, training functions, manipulators here

# Define the relative paths, append it to the system path
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parents[3]
sys.path.append(str(project_root))

# Import shenanigans
from defs.helper_functions.training_functions import *
from defs.helper_functions.data_manipulation_functions import *
from defs.helper_functions.create_model_and_optimizer_functions import *

# Time, to precisely: time
import time

In [3]:
# Set device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")  # Force CPU for testing purposes
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Import and manipulate all the dataset required

df = pd.read_csv(rf"{project_root}\datasets\airfoil_dataset_64pts_200foils.csv") # read csv
df = datareform_tuple_coordinates(df=df, which_coord="y") # take in the y vals
shuffled_df = df.sample(frac=1, random_state=69).reset_index(drop=True) # shuffle the dataframe
input_tensor, target_tensor = seperate_transform(df=df, target_coord=67) # get the target and input tensors
input_tensor = input_tensor.to(device)
target_tensor = target_tensor.to(device)
del df # Clear memory

In [6]:
"""
Trial 1, define configs for Trial 1 here, set device
"""

import torch.nn as nn

cfg_shape_net = {
    'input_dim': 64,
    'output_dim': 3,
    'hidden_units': [512,1024,1024,1024,1024,1024,512],
    'shape_activation': nn.Tanh
}

cfg_param_net = {
    'input_dim': 3,
    'hidden_units': [512,1024,1024,512],
    'param_activation': nn.Tanh
}

NIF_model, NIF_optimizer = create_NIF_Pointwise_and_optimizer(cfg_shape_net=cfg_shape_net, cfg_param_net=cfg_param_net, lr=0.005)
NIF_model = NIF_model.to(device)

In [7]:
# Show the model

from torchviz import make_dot

# Create and move inputs to device
show_tensor = torch.randn(5, cfg_shape_net['input_dim'] + cfg_param_net['input_dim']).to(device)

# Forward pass
output = NIF_model(show_tensor)

make_dot(output, params=dict(NIF_model.named_parameters())).render("model_graph", format="png")

'model_graph.png'

In [8]:
# Now that we have imported data, defined model, and seen what its like, we'll train here


def mse_loss(predictions, targets): return torch.mean((predictions - targets) ** 2)
torch.cuda.synchronize()
start = time.perf_counter()
epoch_train = train_one_epoch(model= NIF_model, optimizer= NIF_optimizer, criterion=mse_loss, inputs= input_tensor, targets= target_tensor, batch_size= 2, device= device, lambda_reg= 0)
torch.cuda.synchronize()
end = time.perf_counter()
print(f"Training took {end - start} seconds")

Rows 0 to 1, loss: 0.0036882534623146057
Rows 2 to 3, loss: 0.02000243589282036
Rows 4 to 5, loss: 0.03447306156158447
Rows 6 to 7, loss: 0.40210187435150146
Rows 8 to 9, loss: 0.2790471315383911
Rows 10 to 11, loss: 0.28552043437957764
Rows 12 to 13, loss: 0.6466492414474487
Rows 14 to 15, loss: 0.708027720451355
Rows 16 to 17, loss: 0.7918782234191895
Rows 18 to 19, loss: 0.8865200281143188
Rows 20 to 21, loss: 0.9690098762512207
Rows 22 to 23, loss: 0.9198931455612183
Rows 24 to 25, loss: 0.8285595178604126
Rows 26 to 27, loss: 0.863127589225769
Rows 28 to 29, loss: 0.7451554536819458
Rows 30 to 31, loss: 0.663211464881897
Rows 32 to 33, loss: 0.6170756220817566
Rows 34 to 35, loss: 0.6055480241775513
Rows 36 to 37, loss: 0.6268877387046814
Rows 38 to 39, loss: 0.6776524782180786
Rows 40 to 41, loss: 0.7556130886077881
Rows 42 to 43, loss: 0.8547343015670776
Rows 44 to 45, loss: 0.9632551074028015
Rows 46 to 47, loss: 1.0577229261398315
Rows 48 to 49, loss: 0.9802230596542358
Rows 5

In [9]:
# Brief trial to ensure the model updates, we will calculate the MSE for the first row of data

start_idx = 2

trial_input = input_tensor[start_idx:start_idx+1].to(device)

result = NIF_model(trial_input)

trial_target = target_tensor[start_idx:start_idx+1].to(device)
trial_mse = mse_loss(result, trial_target).item()
print("Trial MSE for the first input-target pair:")

print(trial_mse)

Trial MSE for the first input-target pair:
0.8288679122924805
